In [128]:
import pandas as pd
messages = pd.read_csv(r"C:\Users\mithr\Downloads\test_twitter_x_test.csv", encoding = 'latin-1')
messages = messages.drop(labels = ["tweet_id", "airline", "name", "retweet_count", "tweet_created", "tweet_location", "user_timezone", ], axis = 1)
messages.dropna(inplace=True)
print(messages)

                                                   text sentiment
0     @AmericanAir In car gng to DFW. Pulled over 1h...  negative
1     @AmericanAir after all, the plane didnât lan...  negative
2     @SouthwestAir can't believe how many paying cu...  positive
3     @USAirways I can legitimately say that I would...  negative
4     @AmericanAir still no response from AA. great ...  negative
...                                                 ...       ...
3655  @USAirways Been stuck for 40+ minutes due to l...  negative
3656  @USAirways 4 hours... 4 hours... FOUR HOURS.  ...  negative
3657  Nice RT @VirginAmerica: The man of steel might...  negative
3658  @AmericanAir Aww Thanks AA..DFW was on GMA up ...  negative
3659  @united the lounge tells us they have no pillo...  negative

[3660 rows x 2 columns]


In [129]:
print(messages.isnull().sum())

text         0
sentiment    0
dtype: int64


In [130]:
messages.columns = ["Text", "Airline_Sentiment"]
print(messages.head(n=10))

                                                Text Airline_Sentiment
0  @AmericanAir In car gng to DFW. Pulled over 1h...          negative
1  @AmericanAir after all, the plane didnât lan...          negative
2  @SouthwestAir can't believe how many paying cu...          positive
3  @USAirways I can legitimately say that I would...          negative
4  @AmericanAir still no response from AA. great ...          negative
5  @united we have developers flying down tmrw mo...          negative
6                  @USAirways hello??? Anyone there?          negative
7  @USAirways @husainhaqqani Mr. Husain u shld pr...          negative
8  @USAirways not likely, flightaware says plane ...          negative
9  @AmericanAir they don't even give an option to...          negative


In [131]:
text = "@john Please review the report."
text = re.sub(r"(@\S+)", "", text).strip()
print(text)

Please review the report.


In [132]:
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [133]:
import re
import nltk
import contractions
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    if not isinstance(text, str):
        return ""
    out = []
    for pattern in [r"(@\S+)", r"(http\S+)", r"(#\S+)"]:
        text = re.sub(pattern, "", text).strip()
    text = contractions.fix(text)
    text = text.lower()
    tokens = word_tokenize(text)
    cleaned_tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens if re.sub(r'[^\w\s]', '', token)]
    tags = pos_tag(cleaned_tokens)
    for token, POS in tags:
        out.append(lemmatizer.lemmatize(token, pos=get_wordnet_pos(POS)))
    out = [
    word for word in out if word.lower() not in stopwords.words('english')]
    return out  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mithr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [134]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

print(lemmatizer.lemmatize("running", pos="v"))

run


In [135]:
messages["Text"] = messages["Text"].apply(preprocess)

In [136]:
print(messages.head(n=10))

                                                Text Airline_Sentiment
0  [car, gng, dfw, pull, 1hr, ago, icy, road, onh...          negative
1  [plane, didnât, land, identical, bad, conditio...          negative
2  [believe, many, pay, customer, leave, high, dr...          positive
3  [legitimately, say, would, rather, drive, cros...          negative
4             [still, response, aa, great, job, guy]          negative
5  [developer, fly, tomorrow, morn, w45, min, lay...          negative
6                                    [hello, anyone]          negative
7  [mr, husain, shld, protest, well, one, party, ...          negative
8  [likely, flightaware, say, plane, still, duran...          negative
9  [even, give, option, hold, say, line, busy, pl...          negative


In [137]:
import gensim.models

sentences = messages["Text"]
model = gensim.models.Word2Vec(sentences=sentences)
vec_length = model.wv.vector_size

In [138]:
import numpy as np
def vectorize(text):
    vector = np.zeros(vec_length)
    for word in text:
        try:
            vector += model.wv[word]
        except:
            pass
    return vector/len(vector)

In [139]:
vectors = []
for sentence in sentences:
    vectors.append(vectorize(sentence))
print(vectors[0])

[-0.00845388  0.01265165  0.00806778 -0.00017832  0.00494695 -0.027983
  0.01307328  0.04203235 -0.01372713 -0.00862439 -0.01507612 -0.02376317
 -0.00554257  0.00385343  0.00387209 -0.01119375 -0.00189941 -0.01310216
 -0.00233838 -0.03182279  0.01013202  0.00908931  0.00360489 -0.00290157
 -0.00680846  0.00545431 -0.01535735 -0.01670377 -0.01611695  0.00614092
  0.01601116  0.00458121  0.00509558 -0.01625881 -0.01127541  0.02407755
 -0.00126698 -0.01630341 -0.00835177 -0.03782517 -0.00261121 -0.01434057
 -0.00602728 -0.00148651  0.01790753 -0.01056967 -0.00900267 -0.00720366
  0.0157942   0.01198621  0.01610245 -0.02243345  0.00624345 -0.00072873
 -0.01586295  0.01314529  0.0042365  -0.00138195 -0.02359788  0.00918357
  0.00696423  0.00759946 -0.00433357 -0.00894069 -0.01866667  0.01883733
  0.01470138  0.01622159 -0.02620173  0.02540401 -0.01141066  0.00809562
  0.02392183 -0.00768246  0.01804566  0.00770411 -0.00309058 -0.00164506
 -0.01518904  0.01177871 -0.00619494 -0.00272496 -0.0

In [140]:
from sklearn.model_selection import train_test_split
messages_train, messages_test, sentiment_train, sentiment_test = train_test_split(vectors, messages["Airline_Sentiment"], train_size=0.8, test_size=0.2, random_state=42)

In [141]:
print(messages.columns)

Index(['Text', 'Airline_Sentiment'], dtype='object')


In [142]:
from sklearn.linear_model import LogisticRegression
clf= LogisticRegression()
clf.fit(messages_train, sentiment_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [143]:
from sklearn.metrics import accuracy_score
sentiment_pred = clf.predict(messages_test)
print("Accuracy:", accuracy_score(sentiment_test, sentiment_pred))

Accuracy: 0.7622950819672131


In [144]:
def predict_tweet_sentiment(model, glove_model, tweet):
    vec_length = glove_model.wv.vector_size
    vector = np.zeros(vec_length)
    for word in tweet:
        try:
            vector+=glove_model.wv[word]
        except KeyError:
            pass
    vector = vector/len(vector)
    vector = vector.reshape(1, -1)
    pred = model.predict(vector)
    return pred.item()

In [152]:
predict_tweet_sentiment(clf, model, "@USAirways your customer service is horrible")

'negative'